# Part 1 

In [1]:
#install.packages("RODBC")
#install.packages("sqldf")


In [2]:
#library(RODBC)
library(sqldf) # Бибилиотека для использования SQL запросов

cars<-read.table("~/Documents/VMK/MaSP/Data/CARS.csv", header=TRUE, sep=",") 

Loading required package: gsubfn

Loading required package: proto

Loading required package: RSQLite



In [3]:
empty_counter <- function(size = 0, names = NA) {
    if(size == 0)
        counter <- vector()
    else{
        counter <- vector("integer", size)
    }
    if(!is.na(names)[1]){ #is.na returns multiple boolean values, that's why '[1]' is used
        names(counter) <- names
    }
    return(counter)
}

In [4]:
MFL <- function(table){
    fl <- sqldf(paste("Select Make, Origin from ",
                      table,
                      " group by Make")) # выделение из таблицы интересующие нас столбцы
    line_1 <- vector() #пустой вектор, который будет хранить символьные значения - элементы первой строки
    for(i in 1:length(fl[[1]])){
        elem <- paste(fl$Make[i],'[', fl[[2]][i], ']') # индексация разная (просто на заметку)
        line_1 <- append(line_1, elem)
    }
    return(line_1)
}


In [5]:
# Make cell for second line function (MCFSL) function
MCFSL <- function(vec) {
    cell <- vector()
    for(i in 1:length(names(vec))){
        elem <- paste(c(names(vec)[i], vec[[i]]), collapse="=")
        cell <- append(cell, elem)
    }
    cell <- paste(cell, collapse=" ")
    return(cell)
}

In [6]:
MSL <- function(table) {
    line_2 <- vector() #пустой вектор, который будет хранить символьные значения - элементы первой строки
    
    names <- levels(factor(table$Type))
    counter <- empty_counter(6, names) # многомерный счетчик для каждой марки машины
    inc <- empty_counter(6, names) # многомерный инкремент для каждой марки машины

    initial_brand <- table[1,]$Make
    for(i in 1:nrow(table)) { # выбираю конкретную марку машины
        car <- table[i,] #
        if(car$Make != initial_brand){
            line_2 <- append(line_2, MCFSL(counter))
            counter <- empty_counter(6, names)
            initial_brand <- car$Make
        }
        inc[car$Type] = 1 
        counter <- counter + inc
        inc[car$Type] = 0
    }
    line_2 <- append(line_2, MCFSL(counter)) # сохраняю данные последнего счетчика, а то без этой строчки проскакиваю
    return(line_2)
}

In [7]:
invoice_converter <- function(invoice, mode='toNumericRubles'){
    if(mode == 'toChar'){
        return(paste(as.character(invoice), ' RUB'))
    }
    else{
        return(60 * as.numeric(sub(',', '.', sub('.', '', invoice))))
    }
}

In [8]:
MTL <- function(table) {
    line_3 <- vector() #пустой вектор, который будет хранить символьные значения - элементы первой строки
    
    names <- levels(factor(table$Type)) # возможные типы кузовов
    type_counter <- empty_counter(6, names) # многомерный счетчик для каждой марки машины
    val_container <- empty_counter() # контейнер для хранения счетов для одинаковых типов кузова одной модели
    max_val <- empty_counter(6, names) # конейнер максимальных счетов для конкретной модели
    inc <- empty_counter(6, names) # многомерный инкремент для каждой марки машины

    initial_brand <- table[1,]$Make
    initial_type <- table[1,]$Type
    initial_max_val <- table[1,]$Invoice
    
    for(i in 1:nrow(table)) { # выбираю конкретную марку машины
        car <- table[i,] #
        if(car$Type != initial_type) {
            max_val[initial_type] <- invoice_converter(max(val_container), mode='toChar')
            initial_type <- car$Type
        }
        
        car <- table[i,] #
        if(car$Make != initial_brand){
            line_3 <- append(line_3, MCFSL(max_val))
            max_val <- empty_counter(6, names)
            val_container <- empty_counter()
            
            initial_brand <- car$Make
        }
        invoice_rubles <- invoice_converter(car$Invoice)
        val_container <- append(val_container, invoice_rubles) # хранит счета для конкретных типов кузовов
                                                               # конкретных марком
    }
    line_3 <- append(line_3, MCFSL(max_val)) # сохраняю данные последнего счетчика, а то без этой строчки проскакиваю
    return(line_3)
}

In [9]:
construct <- function(table){return(data.frame("Manufacturer_Origin" = MFL(deparse(substitute(table))), 
                   "Types" = MSL(table), 
                   "Maximal_invoice" = MTL(table)))}

In [10]:
data <- construct(cars)
write.table(data, file = "MY_DATA.txt", sep = "\t")

# Part 2

In [11]:
MC <- function(data, colname){
    brands_data = data["Manufacturer_Origin"]
    invoice_data = data["Maximal_invoice"]
    column = empty_counter(6)
    for(i in 1:nrow(data)){
        if(grepl(colname, brands_data[i,1])){
            inv <- gsub(' RUB *|[a-z]|[A-Z]|=', "", invoice_data[i,1])
            current_inv <- c(as.numeric(unlist(strsplit(inv, split=" "))))
            column <- column + current_inv
        }
    }
    for(i in 1:length(column)){
        if(column[i] == 0){
            column[i] = '-'
        }
        else{
            
            column[i] <- invoice_converter(column[i], mode='toChar')
        }
    }
    return(column)
}

In [12]:
txt.data <- read.delim("MY_DATA.txt")
types<- gsub("=[[:digit:]]+|=[[:digit:]]+.[[:digit:]]+ |RUB ",
          "",
          txt.data["Maximal_invoice"][1,]) # Ignoring all irrelavant words to make row names
rownames<-unlist(strsplit(types, split=" "))

df <- cbind(data.frame(Types=rownames), 
      Asia=MC(txt.data, 'Asia'), 
      USA=MC(txt.data, 'USA'),
      Europe=MC(txt.data, 'Europe'))

write.csv(df , file='MY_TABLE.csv')

# Part 3

In [13]:
print.MTT<-function(x){
    prop_names <- 'Manufacturer_Origin Types Maximal_invoice'
    if(is.null(names(x))){
        return(cat("ERROR: Object should have names. Try again"))
    }
    else{
        cat("\t\t","--- OVERWRITED PRINT ---","\n\n")
        nr<- length(x)
        for(i in 1:length(x[[1]])){
            if(nr==3){
                cat(x$Manufacturer_Origin[i], x[[2]][i], x$Maximal_invoice[i], sep=' ; ')
            }
            else if(nr==4){
                cat(x[[1]][i], x[[2]][i], x[[3]][i], x[[4]][i], sep=' ; ')
            }
            cat("\n\n")
        }
        #NextMethod("print")
    }
}

In [14]:
do_csv <- function(txt.data, colnames){
    return(cbind(data.frame(Types=rownames), 
      Asia=MC(txt.data, colnames[1]), 
      USA=MC(txt.data, colnames[2]),
      Europe=MC(txt.data, colnames[3])))
}

In [15]:
my.frame <- do_csv(txt.data, c('Asia','USA','Europe'))
class(my.frame) <- 'MTT'

my.frame

		 --- OVERWRITED PRINT --- 

Hybrid ; 2242.62  RUB ; - ; -

Sedan ; 24998.22  RUB ; 22506.3  RUB ; 29411.94  RUB

Sports ; 21372.48  RUB ; 17684.64  RUB ; 30937.98  RUB

SUV ; 20103.36  RUB ; 20362.86  RUB ; 14413.14  RUB

Truck ; 8133.66  RUB ; 16033.08  RUB ; -

Wagon ; 18198.3  RUB ; 12185.04  RUB ; 22371.6  RUB


In [16]:
write.csv <- function(x, ...) UseMethod("write.csv")

write.csv.MTT <- function(x, ...){
    cat("\t\t","--- MTT OBJECT=", paste("'",deparse(substitute(x)),"'"), " SAVED ---","\n\n")
    class(x)<-"data.frame"
    #print(file)
    NextMethod()
    #return(cat("LoL"))
}

write.csv.default <- function(x, ...) {  
    cat("\t\t","--- OVERWRITED WRITE.CSV ---","\n\n")
    utils::write.csv(x, ...)
}

In [17]:
write.csv(my.frame, file='overwrited_write.csv')

		 --- MTT OBJECT= ' my.frame '  SAVED --- 

		 --- OVERWRITED WRITE.CSV --- 

